In [591]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time
from datetime import date
import glob
from keras.models import Sequential
from keras.layers import Dense, Activation

np.random.seed(1337)

today = str(date.today())
end_date = today


path = r'./data/LSTM'  # use your path
allFiles = glob.glob(path + "/2*.csv")

dataset = pd.DataFrame()

for _file in allFiles:
        try:
            t = pd.read_csv(_file, header=0)
            if "LastGame" not in t:
                print("LastGame not in " + _file)
            k = len(t[((t["Final"]==0) & (t["injured"]==0))])
            #if k > 0:
            #print("Found something in: " + _file + " count: " + str(k))
            dataset = pd.concat([dataset,t])
        except:
            print("There was an exception with " + _file)
            pass

        


predict_file = './data/LSTM/2017-02-01.csv'
print("Predict file " + predict_file)
dfPredict = pd.read_csv(predict_file, header=0)

out_file = predict_file.replace("LSTM","final2")
print("Out file " + out_file)

Predict file ./data/LSTM/2017-02-01.csv
Out file ./data/final2/2017-02-01.csv


In [592]:
print(len(dfTrain))
dfTrain.fillna(0.0, inplace=True)
dfTrain.loc[(dfTrain['Final'] == 0),"Final"] = dfTrain[(dfTrain['Final'])==0]['4GameAvg']
dfTrain = dfTrain.dropna()
#dfTrain[(dfTrain["Final"]==0)] = dfTrain[(dfTrain["Final"]==0)][["4GameAvg"]]
print(len(dfTrain))

print(len(dfTest))
dfTest.loc[:,('LastGame','fvalue','Final')].fillna(0.0, inplace=True)
dfTest.loc[(dfTest['Final'] == 0),"Final"] = dfTest[(dfTest['Final'])==0]['4GameAvg']
dfTest = dfTest.dropna()
print(len(dfTest))

8533
8533
300
300


##dfTrain.loc[:,('4GameAvg','LastGame','fvalue','Final')].head(10)
dfTrain.head()

In [593]:
dfTest.columns[pd.isnull(dfTest).any()]

Index([], dtype='object')

In [594]:
print(len(dfPredict))
dfPredict = dfPredict.fillna(0)
print(len(dfPredict))

358
358


In [595]:
scaler = StandardScaler()
scalerY = StandardScaler()
all_features = ['Salary','AvgPointsPerGame','7GameAvg','FloorAvg'
            ,'4GameAvg','value','fvalue','atHome','LastGame','O/U','odds',"dvp","Game0","Game1","Game2",
           "Game2","Game3","Game4","Game5","Game6","Game7","Game8","Game9","Game10"]
features = ["Game10","Game9","Game8","Game7","Game6","Game5","Game4","Game3","Game2","Game1"]


In [596]:
# convert an array of values into a dataset matrix
def create_dataset(_dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(_dataset)):
        a = _dataset.iloc[i][features].values
        dataX.append(a)
        dataY.append(_dataset.iloc[i]["Final"])
    return numpy.array(dataX), numpy.array(dataY)

In [597]:
scaler = StandardScaler()
dataset.fillna(0.0, inplace=True)
dataset.loc[(dataset['Final'] == 0),"Final"] = dataset[(dataset['Final'])==0]['4GameAvg']

dataset[all_features]=scaler.fit_transform(dataset[all_features])

dfTrain = dataset[:-300].copy()
dfTest  =  dataset[-300:].copy()

X_train, y_train = create_dataset(dfTrain)
X_test, y_test = create_dataset(dfTest)

# X_train = scaler.fit_transform(df_train[all_features].values)
# y_train = df_train["Final"]

In [598]:
# X_train_ =  numpy.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [599]:
#df_test = dfTest.copy()
#df_test[all_features] = scaler.transform(df_test[all_features])
#X_test = scaler.transform(df_test[all_features].values)
#y_test = df_test["Final"]

In [600]:
dfTrain.head()

,4GameAvg,7GameAvg,AvgPointsPerGame,Final,FloorAvg,Game0,Game1,Game10,Game2,Game3,...,RF,Salary,atHome,dvp,fvalue,injured,odds,penalty,teamAbbrev,value
0,2.996755,4.184127,3.965650,73.25,4.261886,3.781048,4.275414,3.020246,1.708908,1.234895,...,0.0,4.904900,1.00034,0.910483,-0.104460,0,-0.428510,0.0,OKC,4.904900
1,2.772739,2.599979,2.683915,42.50,2.997321,1.733374,1.785623,2.223574,2.713562,2.811195,...,0.0,3.129784,-0.99966,0.255539,-0.125888,0,0.711259,0.0,Was,3.129784
2,2.446303,2.890421,2.257830,55.25,2.578318,2.582409,3.495280,1.647259,1.675420,2.341659,...,0.0,2.671689,1.00034,0.910483,0.216955,0,-0.855924,0.0,Min,2.671689
3,1.861710,2.094040,2.193408,36.25,2.113090,1.317180,0.656918,1.172646,1.055883,2.526119,...,0.0,2.442642,1.00034,-1.143660,0.409803,0,-0.357275,0.0,Chi,2.442642
4,1.530441,1.809394,1.884794,41.00,2.198183,1.633487,1.852018,1.240448,1.055883,1.285202,...,0.0,2.213595,1.00034,0.553241,-0.540155,0,-0.713453,0.0,Ind,2.213595


In [601]:
#X_train[features] = scaler.fit_transform(X_train[features])



In [602]:
dataX[-1:]

[([46.5, 35.0, 21.0, 25.25, 29.0, 26.25, 27.0, 41.5, 30.0, 23.5], -2.5)]

In [603]:
df_test.describe()

,4GameAvg,7GameAvg,AvgPointsPerGame,Final,FloorAvg,Game0,Game1,Game10,Game2,Game3,...,O/U,RF,Salary,atHome,dvp,fvalue,injured,odds,penalty,value
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,...,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.0,300.000000
mean,-0.274660,-0.304468,-0.290675,14.068214,-0.323343,-0.223181,-0.226570,-0.257407,-0.249217,-0.203283,...,0.158339,14.041627,-0.342001,0.020340,-0.072132,-0.009821,0.140000,-0.002284,0.0,-0.342001
std,0.675780,0.668852,0.647743,9.662702,0.606105,0.689778,0.765041,0.755124,0.768002,0.730513,...,0.262659,7.917343,0.489413,1.001471,0.988979,1.063695,0.347567,0.905379,0.0,0.489413
min,-1.353549,-1.410584,-1.475929,-1.500000,-1.159111,-1.196631,-1.152330,-1.234319,-1.171099,-1.179862,...,-0.347922,0.000000,-0.706759,-0.999660,-1.709293,-1.411546,0.000000,-2.138165,0.0,-0.706759
25%,-0.843278,-0.821952,-0.774587,6.250000,-0.843492,-0.817895,-0.919949,-0.946161,-0.869703,-0.861248,...,-0.068188,7.640000,-0.706759,-0.999660,-0.875728,-0.841928,0.000000,-0.499746,0.0,-0.706759
50%,-0.289084,-0.325250,-0.332587,13.437500,-0.409578,-0.289328,-0.355597,-0.302044,-0.417609,-0.299482,...,0.159096,13.823000,-0.592235,1.000340,-0.086818,-0.118745,0.000000,-0.143568,0.0,-0.592235
75%,0.239505,0.180005,0.168020,19.766000,0.106274,0.151837,0.308347,0.312410,0.235416,0.379668,...,0.263997,19.980500,-0.134140,1.000340,0.709534,0.709790,0.000000,0.497552,0.0,-0.134140
max,1.394036,1.381056,1.363901,55.500000,1.429861,2.599057,2.416370,2.071020,2.378677,1.704431,...,0.561215,36.106000,1.011096,1.000340,1.744048,4.188210,1.000000,2.135972,0.0,1.011096


In [622]:
print(X_train.shape[1])
_trainX = numpy.reshape(X_train, (X_train.shape[0], 1 ,10))
_trainX[:3]

10


array([[[3.0202458985588412, 3.9642355125021829, 3.9843099978361507,
         3.5784778576584211, 3.8568682203431432, 2.9679541794040043,
         3.1456028688261157, 1.2348949508322926, 1.7089080849492588,
         4.275413972498475]],

       [[2.2235744400767921, 2.1396867290232446, 2.36020522247517,
         3.9689299781754865, 1.3252411732223233, 2.3272064782332915,
         2.3064270868768233, 2.8111947762907366, 2.7135616003009293,
         1.7856231903116739]],

       [[1.6472589169195646, 4.2882208105031152, 1.3515506777772974,
         2.4920023918718037, 1.5108938233445168, 1.1974671103796664,
         0.96374583575795503, 2.3416586580690724, 1.6754196344375365,
         3.4952795274132771]]], dtype=object)

In [648]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# create and fit the LSTM network
trainX2 = numpy.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
testX2 = numpy.reshape(X_test, (X_test.shape[0],1, X_test.shape[1]))
trainY2 = y_train
testY2 = y_test

features = ["Game10","Game9","Game8","Game7","Game6","Game5","Game4","Game3","Game2","Game1"]
batch_size = 1
look_back = len(features)
model2 = Sequential()
model2.add(LSTM(6, batch_input_shape=(batch_size, 1, look_back)))
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.fit(trainX2, y_train, nb_epoch=10, batch_size=batch_size, verbose=2)
# make predictions

Epoch 1/10
65s - loss: 235.9812
Epoch 2/10
64s - loss: 95.9321
Epoch 3/10
63s - loss: 79.1505
Epoch 4/10
64s - loss: 72.5000
Epoch 5/10
66s - loss: 69.2088
Epoch 6/10
64s - loss: 67.4185
Epoch 7/10
63s - loss: 66.3411
Epoch 8/10
66s - loss: 65.6448
Epoch 9/10
69s - loss: 65.2293
Epoch 10/10
96s - loss: 64.9331


In [651]:
trainPredict_ = model2.predict(trainX2, batch_size=batch_size)
testPredict_ = model2.predict(testX2  , batch_size=batch_size)

In [653]:
print ("\naccuracy", np.sum(abs(trainPredict_.T-y_train)<10) / float(len(y_train)))
print ("\naccuracy", np.sum(abs(testPredict_.T-y_test)<10) / float(len(y_test)))


accuracy 0.815305285363

accuracy 0.843333333333


In [605]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# create and fit the LSTM network
trainX = numpy.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
testX = numpy.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
trainY = y_train
testY = y_test

features = ["Game10","Game9","Game8","Game7","Game6","Game5","Game4","Game3","Game2","Game1"]
batch_size = 1
look_back = len(features)
model = Sequential()
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train_, trainY, nb_epoch=3, batch_size=batch_size, verbose=2)
# make predictions

Epoch 1/5
210s - loss: 222.3277
Epoch 2/5
202s - loss: 100.9563
Epoch 3/5
207s - loss: 81.3694
Epoch 4/5
224s - loss: 75.5729
Epoch 5/5
214s - loss: 73.1969


In [606]:
# make predictions
trainPredict = model.predict(trainX, batch_size=batch_size)

In [607]:
testPredict = model.predict(testX[features],batch_size=batch_size)

/Users/marek5050/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [638]:
# print(trainPredict[:5])
# print(testPredict[:5])
# print(y_test[:5])
# print(y_train[:5])
# print(y_test.values)
print(len(trainPredict))
print(len(testPredict))
print(len(y_train))
print(len(y_test))
# print((testPredict.T-y_test.values))
print ("\naccuracy", np.sum(abs(trainPredict.T-y_train)<5) / float(len(y_train)))
print ("\naccuracy", np.sum(abs(testPredict.T-y_test)<5) / float(len(y_test)))

8533
300
8533
300

accuracy 0.578928864409

accuracy 0.613333333333


In [ ]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=20,
#                               random_state=1,
                              verbose=False,
                              warm_start=True)
modelRFR = modelRFR.fit(X_train, y_train)

yRF_prediction = modelRFR.predict(X_test)
print (modelRFR.score(  X_train , y_train ) , modelRFR.score(  X_test , y_test  ))
print ("\naccuracy", np.sum(abs(yRF_prediction-y_test)<5) / float(len(y_test)))
#51.8 at 120

In [ ]:
from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit( X_train , y_train )
print (svr_rbf.score(  X_train , y_train ) , svr_rbf.score(  X_test , y_test  ))
SVR_prediction = svr_rbf.predict(X_test)
print ("\naccuracy", np.sum(abs(SVR_prediction-y_test)<5) / float(len(y_test)))

In [ ]:
#74 63
from sklearn.tree import DecisionTreeRegressor
regr = DecisionTreeRegressor(max_depth=5)
regr.fit( X_train , y_train )
print (regr.score(  X_train , y_train ) , regr.score(  X_test , y_test  ))
DTR_prediction = regr.predict(X_test)
print ("\naccuracy", np.sum(abs(DTR_prediction-y_test)<5) / float(len(y_test)))

In [ ]:
#70 63
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=7)
knr.fit( X_train , y_train )
print (knr.score(  X_train , y_train ) , knr.score(  X_test , y_test  ))
KNR_prediction = knr.predict(X_test)
print ("\naccuracy", np.sum(abs(KNR_prediction-y_test)<5) / float(len(y_test)))

In [ ]:
#74 64
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()
mlp.fit( X_train , y_train )
print (mlp.score(  X_train , y_train ) , mlp.score(  X_test , y_test  ))
mlp_prediction = mlp.predict(X_test)
print ("\naccuracy", np.sum(abs(mlp_prediction-y_test)<5) / float(len(y_test)))

In [ ]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor\
    (activation='identity',solver='adam', learning_rate='adaptive', hidden_layer_sizes=(3, 6))
mlp.fit( X_train , y_train )
print (mlp.score(  X_train , y_train ) , mlp.score(  X_test , y_test  ))
mlp_prediction = mlp.predict(X_test)
print ("\naccuracy", np.sum(abs(mlp_prediction-y_test)<5) / float(len(y_test)))

In [ ]:
# 0.3911
#y_tree_prediction
#pd.concat([dfTest["Name"],pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(16, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
model.compile(optimizer='adam', metrics=['accuracy','recall','fmeasure'],loss="mae")
# MAE 58.23%<5 16-8-1 50 Adam MAE
# MAE 55.69%<5 16-8-1  50 epoch SGD  MAE
# MSE 81% 
# MAPE 35% 
# MSLE 53% 
# HINGE 34% 
# SGD MSE 150 70% 
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=10, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))
model.evaluate(X_test,y_test)
y_predict = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_predict.T[0]-y_test)<5) / float(len(y_test)))
print(y_predict[:5])
#19% 3 layer

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(8, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
model.compile(optimizer='adam', metrics=['accuracy','recall','fmeasure'],loss="mse")
# MAE 58.23%<5 16-8-1 50 Adam MAE
# MAE 55.69%<5 16-8-1  50 epoch SGD  MAE
# MSE 81% 
# MAPE 35% 
# MSLE 53% 
# HINGE 34% 
# SGD MSE 150 70% 
model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=10, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))
model.evaluate(X_test,y_test)
y_predict = model.predict(X_test)
print ("\naccuracy", np.sum(abs(y_predict.T[0]-y_test)<5) / float(len(y_test)))
print(y_predict[:5])
#19% 3 layer

In [ ]:
kk = pd.DataFrame.from_dict(data= {"NN":y_predict.T[0] ,"RF":yRF_prediction})
kk.head()

In [ ]:
X_test

In [ ]:
# df_test["NN"] = kk["NN"]
# df_test["RF"] = kk["RF"]
df_test[["NN","RF"]] = None
df_test[["NN", "RF"]] = kk
df_test.head()

In [ ]:
output = pd.concat([df_test,
            pd.DataFrame(y_predict.T[0],columns=["RF"]),
            pd.DataFrame(yRF_prediction,columns=["NN"])]
          ,axis=1)
#output.to_csv('data/output/2017-01-24.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [ ]:
output.head()

In [ ]:
output.loc[:,("Name","NN","RF","AvgPointsPerGame","Final","O/U","odds","dvp")].sort_values(by=["O/U","NN","dvp"], ascending=False).head(10)

##  Predictions for Today!!!

In [ ]:
scaler1 = StandardScaler()
features = ['Salary','AvgPointsPerGame','7GameAvg','FloorAvg'
            ,'4GameAvg','value','fvalue','atHome','LastGame','O/U','odds',"dvp"]

X_predict = scaler1.fit_transform(dfPredict[features].values)
y_tree_prediction = modelRFR.predict(X_predict) ## Random Forest
y_predict = model.predict(X_predict) ## Neural Networks prediction

In [ ]:
output = pd.concat([dfPredict,
            pd.DataFrame(y_predict,columns=["NN"]),
            pd.DataFrame(y_tree_prediction,columns=["RF"])]
          ,axis=1)

In [ ]:
#y_predict
#output.loc[:,"NN"].head()
#print(out_file)

In [ ]:
output.to_csv(predict_file, sep=',', encoding='utf-8', index=False, float_format='%.3f')
withoutInjuries = output[(output["injured"]==0)]
withoutInjuries.head()
withoutInjuries.to_csv(out_file, sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [ ]:
from sklearn.metrics import mean_squared_error
rms = np.sqrt(mean_squared_error(y_test, y_predict))

In [ ]:
print(rms)

In [ ]:
import pandas as pd 
import numpy as np
predicted_file = 'data/final/2017-01-28.csv'
print("Predict file " + predicted_file)
dfPredicted = pd.read_csv(predicted_file, header=0)
dfPredicted["ErrorNN"] = dfPredicted["NN"].T-dfPredicted["Final"]
dfPredicted["ErrorRF"] = dfPredicted["RF"].T-dfPredicted["Final"]
#rmsNN = np.sqrt(mean_squared_error(dfPredicted["Final"], dfPredicted["NN"]))
#rmsRF = np.sqrt(mean_squared_error(dfPredicted["Final"], dfPredicted["RF"]))
dfPredicted.iloc[:,-3:].describe()
print(np.sum(abs(dfPredicted["NN"].T-dfPredicted["Final"]) < 5)/float(len(dfPredicted["Final"])))
print(np.sum(abs(dfPredicted["RF"].T-dfPredicted["Final"]) < 5)/float(len(dfPredicted["Final"])))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorNN"]) < 5)]
print("Len NN > 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorNN"]) > 5)]
print("Len NN < 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorRF"]) < 5)]
print("Len RF > 5 " + str(len(dfPredictedFiltered)))
dfPredictedFiltered = dfPredicted[(abs(dfPredicted["ErrorRF"]) > 5)]
print("Len RF < 5 " + str(len(dfPredictedFiltered)))

dfPredictedFiltered.loc[:,("Name","NN","RF","Final","ErrorNN","ErrorRF")]

In [ ]:
[1,2,5,2,9,10,13,99]
smallest = 1
diff. = 2

In [314]:
k=np.arange(20).reshape((5,1,4))
mm = np.array(1)
pp = []
for i in range(len(k)): 
        pp.append([k[i],i])
nn = np.array(pp)

In [315]:
nn

array([[array([[0, 1, 2, 3]]), 0],
       [array([[4, 5, 6, 7]]), 1],
       [array([[ 8,  9, 10, 11]]), 2],
       [array([[12, 13, 14, 15]]), 3],
       [array([[16, 17, 18, 19]]), 4]], dtype=object)

TypeError: list() takes at most 1 argument (3 given)

In [365]:
class B():
    def lum(self):
        print("Lum!")

In [366]:
B.lum

<function __main__.B.lum>

In [368]:
B.lum(B())

Lum!
